In [1]:
print("Hello World")

Hello World


In [16]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredCSVLoader
from dotenv import load_dotenv
import os

In [17]:
load_dotenv()

True

In [18]:
llm = ChatGroq(model="llama3-8b-8192", 
               api_key=os.getenv('GROQ_API_KEY'),
               temperature=0.5,
               max_tokens=200)

In [19]:
llm.invoke("Hi how are you").content

"I'm just a language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have! How about you, how's your day going?"

In [20]:
template = ChatPromptTemplate.from_messages([
    ("system", """You are a medical AI chatbot. Users ask you medical questions, and you provide accurate and in a simple way answers based on your training data.

    Prompt: What is the definition of [medical term]?
    How does [medication] work?
    What are the side effects of [treatment]?"""),
    ("human", "{input} {agent_scratchpad}")
])

In [21]:
tools = []
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [22]:
agent_executor.invoke({"input": "Hi my name is arbaz khan"})



> Entering new AgentExecutor chain...
Nice to meet you, Arbaz! I'm a medical AI chatbot here to help answer your questions and provide you with accurate and reliable information about various medical topics. What's on your mind? Do you have a specific question or topic you'd like to discuss?

> Finished chain.


{'input': 'Hi my name is arbaz khan',
 'output': "Nice to meet you, Arbaz! I'm a medical AI chatbot here to help answer your questions and provide you with accurate and reliable information about various medical topics. What's on your mind? Do you have a specific question or topic you'd like to discuss?"}

In [41]:
import pandas as pd
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# Load the CSV file with questions and answers
data = pd.read_csv("Test.csv")

# Convert the CSV data into LangChain documents
documents = [Document(page_content=row['Answer'], metadata={'Question': row['Question']}) for _, row in data.iterrows()]

# Initialize an embeddings model and vector store (we'll use FAISS here)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

# Initialize the RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

# Prompt user for a question and get the answer from the CSV data
def chat_with_bot(question):
    return qa_chain.run(question)

# Example usage
user_question = "What does cartilage link?"
answer = chat_with_bot(user_question)
print("Bot:", answer)

C:\Users\Arbaz Khan\AppData\Local\Temp\ipykernel_2876\4596074.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Arbaz Khan\Desktop\LangChain\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [15]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pandas as pd

In [10]:
df = pd.read_csv("Test.csv", header=0)

In [11]:
df.head()

,question,answer
0,Glycogen is the way the body stores glucose - ...,Sugar
1,How many square feet is the average adult huma...,19 feet
2,What is the largest artery in the body?,The aorta
3,What body system is the liver a part of?,Digestive System
4,What does cartilage link?,Joints and bones


In [12]:
texts = [f"Q: {q}\nA: {a}" for q, a in zip(df['question'], df['answer'])]

In [13]:
texts

['Q: Glycogen is the way the body stores glucose - better known by what common name?\nA: Sugar',
 'Q: How many square feet is the average adult human’s skin?\nA: 19 feet',
 'Q: What is the largest artery in the body?\nA: The aorta',
 'Q: What body system is the liver a part of?\nA: Digestive System',
 'Q: What does cartilage link?\nA: Joints and bones']

In [25]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_texts(texts, embeddings)

In [38]:
db.similarity_search("square feet")

[Document(page_content='Q: How many square feet is the average adult human’s skin?\nA: 19 feet'),
 Document(page_content='Q: What does cartilage link?\nA: Joints and bones'),
 Document(page_content='Q: What is the largest artery in the body?\nA: The aorta'),
 Document(page_content='Q: What body system is the liver a part of?\nA: Digestive System')]

In [32]:
texts = [f"Q: {q}\nA: {a}" for q, a in zip(df['question'], df['answer'])]

@tool
def get_knowledge(question: str) -> str:
    """
    Provide answer by finding the answer from the database
    """
    global db
    return db.similarity_search(question)[0].page_content

@tool
def add_knowledge(question: str, answer: str) -> None:
    """
    Add the question and answer to the database if not provided
    """
    global db
    texts.append(f'Q: {question}\nA: {answer}')
    db = FAISS.from_texts(texts, embeddings)
    return 

In [33]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an helpfull ai agent which follows the below sequence to answer the question:
     -> Use get_knowledge tool to get the answer by sending the question as a perameter
     -> If can't find the answer then use add_knowledge tool to add the question and answer"""),
     ("human", "{input}{agent_scratchpad}")
])

In [34]:
llm = ChatGroq(model="llama3-8b-8192", temperature=0.5)
tools = [get_knowledge, add_knowledge]
agent = create_tool_calling_agent(
    llm= llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools)